# EP 4: Create Your Own Gazebo Simulation for RobotArm

<img src="images/simpose_home.png"/>

We would like to test loads of things in our Real Robot Arm, but there is a problem: What if we break it? What if we dont have the robot arm available all the time because we work in a team or because I'm on the go? What if I want to test something risky? What if I dont have some sensor for perception and I want to start coding to test if it would work before buying it?.

For all those reasons, simulation is vital for the robot software development.

Objectives:
* Learn how to create your own simulation of your robot arm
* Integrate everything done until now with Moveit
* Prepare simulation for integrating perception in Moveit

## Create the launch files for the simulation

We need to start three things:
* The Gazebo world.
* Spawn the URDF of the robot arm
* Start the control system for the simulated robot arm.

In [ ]:
roscd openmanipulator_morpheus_chair_tutorials
mkdir launch
mkdir worlds
mkdir config
touch launch/start_simulation.launch
touch worlds/empty_kinect.world
touch launch/trajectory_controller.launch
touch config/trajectory_controller.yaml

<p style="background:#3B8F10;color:white;" id="simple_python">**start_simulation.launch** </p>

In [ ]:
<launch>
  <!-- These are the arguments you can pass this launch file, for example paused:=true -->
  <arg name="paused" default="false"/>
  <arg name="use_sim_time" default="true"/>
  <arg name="gui" default="true"/>
  <arg name="headless" default="false"/>
  <arg name="debug" default="false"/>

  <!-- We resume the logic in empty_world.launch, changing only the name of the world to be launched -->
  <include file="$(find gazebo_ros)/launch/empty_world.launch">
    <arg name="world_name" value="$(find openmanipulator_morpheus_chair_tutorials)/worlds/empty_kinect.world"/>
    <arg name="debug" value="$(arg debug)" />
    <arg name="gui" value="$(arg gui)" />
    <arg name="paused" value="$(arg paused)"/>
    <arg name="use_sim_time" value="$(arg use_sim_time)"/>
    <arg name="headless" value="$(arg headless)"/>
  </include>

  <!-- Load the URDF into the ROS Parameter Server -->
  <param name="robot_description"
   command="$(find xacro)/xacro --inorder '$(find open_manipulator_support_description)/urdf/open_manipulator_support.urdf.xacro'"/>

  <!-- Run a python script to the send a service call to gazebo_ros to spawn a URDF robot -->
  <node name="urdf_spawner" pkg="gazebo_ros" type="spawn_model" respawn="false" output="screen"
     args="-urdf -model assem_urdf10_description -z 0.02 -param robot_description"/>

  <node name="tf_footprint_base" pkg="tf" type="static_transform_publisher" args="0 0 0 0 0 0 base_link base_footprint 40" />

  <node args="0.85931 0.242068 1.04639 2.1 0 -1.57 /base_link /camera_depth_optical_frame 20" name="kinect_base_link" pkg="tf" type="static_transform_publisher" />

  <!-- ros_control robotis manipulator launch file -->

  <include file="$(find openmanipulator_morpheus_chair_tutorials)/launch/trajectory_controller.launch"/>

</launch>


<p style="background:#3B8F10;color:white;" id="simple_python">**END start_simulation.launch** </p>

<p style="background:#3B8F10;color:white;" id="simple_python">**empty_kinect.world** </p>

In [ ]:
<sdf version="1.4">
  <world name="default">
    <!-- A global light source -->
    <include>
      <uri>model://sun</uri>
    </include>

    <!-- A ground plane -->
    <include>
      <uri>model://ground_plane</uri>
    </include>

    <physics type="ode">
      <real_time_update_rate>1000.0</real_time_update_rate>
      <max_step_size>0.001</max_step_size>
      <real_time_factor>1</real_time_factor>
      <ode>
        <solver>
          <type>quick</type>
          <iters>150</iters>
          <precon_iters>0</precon_iters>
          <sor>1.400000</sor>
          <use_dynamic_moi_rescaling>1</use_dynamic_moi_rescaling>
        </solver>
        <constraints>
          <cfm>0.00001</cfm>
          <erp>0.2</erp>
          <contact_max_correcting_vel>2000.000000</contact_max_correcting_vel>
          <contact_surface_layer>0.01000</contact_surface_layer>
        </constraints>
      </ode>
    </physics>

    <scene>
      <ambient>0.4 0.4 0.4 1</ambient>
      <background>0.7 0.7 0.7 1</background>
      <shadows>true</shadows>
    </scene>

    <gui fullscreen='0'>
      <camera name='user_camera'>
        <pose> 2.0 -2.0 1.2 0.0 0.275643 2.356190 </pose>
        <view_controller>orbit</view_controller>
      </camera>
    </gui>
    

    <model name='kinect'>
      <static>1</static>
      <pose frame=''>0.85931 0.242068 0.805 0 0.523599 -2.6122</pose>
      <link name='link'>
        <inertial>
          <mass>0.1</mass>
          <inertia>
            <ixx>1</ixx>
            <ixy>0</ixy>
            <ixz>0</ixz>
            <iyy>1</iyy>
            <iyz>0</iyz>
            <izz>1</izz>
          </inertia>
        </inertial>
        <collision name='collision'>
          <geometry>
            <box>
              <size>0.073 0.276 0.072</size>
            </box>
          </geometry>
          <max_contacts>10</max_contacts>
          <surface>
            <contact>
              <ode/>
            </contact>
            <bounce/>
            <friction>
              <torsional>
                <ode/>
              </torsional>
              <ode/>
            </friction>
          </surface>
        </collision>
        <visual name='visual'>
          <geometry>
            <mesh>
              <uri>model://kinect/meshes/kinect.dae</uri>
              <scale>1 1 1</scale>
            </mesh>
          </geometry>
        </visual>
        <sensor name='camera' type='depth'>
          <update_rate>20</update_rate>
          <camera name='__default__'>
            <horizontal_fov>1.0472</horizontal_fov>
            <image>
              <width>640</width>
              <height>480</height>
              <format>R8G8B8</format>
            </image>
            <clip>
              <near>0.05</near>
              <far>3</far>
            </clip>
          </camera>
          <plugin filename="libgazebo_ros_openni_kinect.so" name="kinect_camera_controller">                                                                            
          <cameraName>camera</cameraName>                                                                                                                             
          <alwaysOn>true</alwaysOn>                                                                                                                                   
          <updateRate>10</updateRate>                                                                                                                                 
          <imageTopicName>rgb/image_raw</imageTopicName>                                                                                                              
          <depthImageTopicName>depth/image_raw</depthImageTopicName>                                                                                                  
          <pointCloudTopicName>depth/points</pointCloudTopicName>                                                                                                     
          <cameraInfoTopicName>rgb/camera_info</cameraInfoTopicName>                                                                                                  
          <depthImageCameraInfoTopicName>depth/camera_info</depthImageCameraInfoTopicName>                                                                            
          <frameName>camera_depth_optical_frame</frameName>                                                                                                           
          <baseline>0.1</baseline>                                                                                                                                    
          <distortion_k1>0.0</distortion_k1>                                                                                                                          
          <distortion_k2>0.0</distortion_k2>                                                                                                                          
          <distortion_k3>0.0</distortion_k3>                                                                                                                          
          <distortion_t1>0.0</distortion_t1>                                                                                                                          
          <distortion_t2>0.0</distortion_t2>                                                                                                                          
          <pointCloudCutoff>0.4</pointCloudCutoff>                                                                                                                    
          <robotNamespace>/</robotNamespace>                                                                                                                          
        </plugin>
        </sensor>
        <self_collide>0</self_collide>
        <kinematic>0</kinematic>
      </link>
    </model>
    <model name='kinect_pilar'>
      <static>1</static>
      <pose frame=''>0.89 0.242 0.5 0 -0 0</pose>
      <link name='link'>
        <inertial>
          <mass>1</mass>
          <inertia>
            <ixx>0.145833</ixx>
            <ixy>0</ixy>
            <ixz>0</ixz>
            <iyy>0.145833</iyy>
            <iyz>0</iyz>
            <izz>0.125</izz>
          </inertia>
        </inertial>
        <collision name='collision'>
          <geometry>
            <cylinder>
              <radius>0.04</radius>
              <length>0.5</length>
            </cylinder>
          </geometry>
          <max_contacts>10</max_contacts>
          <surface>
            <contact>
              <ode/>
            </contact>
            <bounce/>
            <friction>
              <torsional>
                <ode/>
              </torsional>
              <ode/>
            </friction>
          </surface>
        </collision>
        <visual name='visual'>
          <geometry>
            <cylinder>
              <radius>0.04</radius>
              <length>0.5</length>
            </cylinder>
          </geometry>
          <material>
            <script>
              <uri>file://media/materials/scripts/gazebo.material</uri>
              <name>Gazebo/Metal</name>
            </script>
            <ambient>1 1 1 1</ambient>
          </material>
        </visual>
        <self_collide>0</self_collide>
        <kinematic>0</kinematic>
      </link>
    </model>

  </world>
</sdf>


<p style="background:#3B8F10;color:white;" id="simple_python">**END empty_kinect.world** </p>

<p style="background:#3B8F10;color:white;" id="simple_python">**trajectory_controller.launch** </p>

In [ ]:
<launch>

  <!-- Load joint controller configurations from YAML file to parameter server -->
  <rosparam file="$(find openmanipulator_morpheus_chair_tutorials)/config/trajectory_controller.yaml" command="load"/>

  <!-- load the controllers -->
  <node name="controller_spawner" pkg="controller_manager" type="spawner" respawn="false"
    output="screen" args="joint_state_controller arm"/>

  <!-- convert joint states to TF transforms for rviz, etc -->
  <node name="robot_state_publisher" pkg="robot_state_publisher" type="robot_state_publisher"
    respawn="false" output="screen">
  </node>

</launch>

<p style="background:#3B8F10;color:white;" id="simple_python">**END trajectory_controller.launch** </p>

<p style="background:#3B8F10;color:white;" id="simple_python">**trajectory_controller.yaml** </p>

In [ ]:
joint_state_controller:
    type: joint_state_controller/JointStateController
    publish_rate: 50

arm:
    type: position_controllers/JointTrajectoryController
    joints:
        - id_1
        - id_2
        - id_3
        - id_4
        - id_5
        - id_6

    constraints:
        goal_time: 0.6
        stopped_velocity_tolerance: 0.05
        id_1: {trajectory: 0.1, goal: 0.1}
        id_2: {trajectory: 0.1, goal: 0.1}
        id_3: {trajectory: 0.1, goal: 0.1}
        id_4: {trajectory: 0.1, goal: 0.1}
        id_5: {trajectory: 0.1, goal: 0.1}
        id_6: {trajectory: 0.1, goal: 0.1}

    stop_trajectory_duration: 0.5
    state_publish_rate:  25
    action_monitor_rate: 10


<p style="background:#3B8F10;color:white;" id="simple_python">**END trajectory_controller.yaml** </p>

In [ ]:
cd open_manipulator_support_description/urdf
ls open_manipulator.gazebo.xacro

In [ ]:
  <gazebo>
    <plugin name="gazebo_ros_control" filename="libgazebo_ros_control.so">
      <robotNamespace></robotNamespace>
      <controlPeriod>0.001</controlPeriod>
      <robotSimType>gazebo_ros_control/DefaultRobotHWSim</robotSimType>
    </plugin>
  </gazebo>

In [ ]:
cd open_manipulator_support_description/urdf
ls open_manipulator_support.urdf.xacro

In [ ]:
<transmission name="tran6">
    <type>transmission_interface/SimpleTransmission</type>
    <joint name="id_6">
      <hardwareInterface>hardware_interface/PositionJointInterface</hardwareInterface>
    </joint>
    <actuator name="motor6">
      <hardwareInterface>hardware_interface/PositionJointInterface</hardwareInterface>
      <mechanicalReduction>1</mechanicalReduction>
    </actuator>
  </transmission>

## Launch the simulation

Now you are ready to launch the simulation. You can do it through the **ROSDS Simulations Panel** if you want the easy way.
Or through the WebShell through the command:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in ROSDS WebShell #1</p>
</th>
</tr>
</table>
<br>

In [ ]:
roslaunch openmanipulator_morpheus_chair_tutorials start_simulation.launch

Or through the **Simulations Pannel**:

<img src="images/launch1.png"/>

<img src="images/launch2.png"/>

You should now have a simulation like this:

<img src="images/simmanipulator.png"/>

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in ROSDS WebShell #2</p>
</th>
</tr>
</table>
<br>

In [ ]:
roslaunch openmanipulator_ep2_movit_config open_manipulator_planning_execution.launch

And be able to execute Moveit! movements and that the simulations moves accordingly:

<img src="images/movit_sim.png"/>